# Assignment 1: Part Of Speech tagging

## Notebook Summary and Structure
The notebook is divided as follows:

1. **Data Pipeline**  
    1.1 **Data loading**: load/download the nltk corpus  
    1.2 **GloVe loading**: load/download the GloVe embeddings  
    1.3 **Data visualization**: visualize the dataset in a human-readable way  
    1.4 **Data cleanup**: clear the data from errors, handle capitalized words  
    1

## Imports

In [ ]:
# Main framework
from tensorflow import keras

# Data packages
import numpy as np
import pandas as pd

# System packages
import glob
import random

# File management
import requests
import zipfile
import io
import pathlib
import os

# To store vocabulary as .json
!pip install simplejson
import simplejson

# Notebook visualization
from IPython.core.display import display, HTML

# Seed initialization
random.seed(0)

# Typing
from typing import Set

# For GloVe wrapper
import gensim
from gensim import downloader as gensloader
from gensim.models.keyedvectors import KeyedVectors

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 1 - Data Pipeline

### 1.1 - Data loading
First we load the nltk corpus (downloading it if not present), and store it into a dataframe.

In [ ]:
DATASET_PATH = "./dependency_treebank"  # Change if dataset already present locally
DATASET_URL = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"


def load_dataset(ds_path: str, ds_url: str) -> pd.DataFrame:
    # Check if dataset is already present, otherwise download it
    if not pathlib.Path(ds_path).exists():
        request_zip = requests.get(ds_url, stream=True)
        zip = zipfile.ZipFile(io.BytesIO(request_zip.content))
        zip.extractall()

    # Load each file into a list
    documents = []
    for file_name in sorted(glob.glob(f"{ds_path}/*.dp")):
        with open(file_name) as f:
            documents.append(f.read())

    # Convert each row of the documents into a list
    raw_df = []
    sentence_idx = 0
    for doc_idx, doc in enumerate(documents):
        rows = doc.split('\n')
        for row in rows:
            cols = row.split('\t')[:2]  # Ignore the last column
            if cols == ['']:
                sentence_idx += 1
            else:
                raw_df.append([doc_idx, sentence_idx, *cols])

    # Finally, convert the nested list into a pandas dataframe
    df = pd.DataFrame(raw_df, columns=['document', 'sentence', 'token', 'tag'])
    return df


dataset = load_dataset(DATASET_PATH, DATASET_URL)
dataset.head()

### 1.2 - GloVe loading
Now we load the GloVe embeddings (if present locally) or download them.

In [ ]:
GLOVE_TYPE = 'glove-wiki-gigaword-50'
GLOVE_FILE = './glove/glove-wiki-gigaword-50.kv'


def load_glove(gl_file: str, gl_type: str) -> KeyedVectors:
    # Load local version
    path = pathlib.Path(gl_file)
    if path.exists():
        return gensim.models.KeyedVectors.load(gl_file)

    # Otherwise download and store glove
    path.parent.mkdir(parents=True, exist_ok=True)
    glove = gensloader.load(gl_type)
    glove.save(gl_file)
    return glove


glove_embedding = load_glove(GLOVE_FILE, GLOVE_TYPE)

### 1.3 - Data visualization
One of the most important ML tasks is to gain familiarity with the data, to gain a deeper insight on the dataset. 

To do so, we define a function that displays a series of tokens and its POS tagging (predicted or from the corpus) in a more human-readable way.

In [ ]:
# NOTE: this could be put in its own file to keep things clean,
#       but to upload just one notebook we instead code-golfed a bit :)

# Define a mapping between POS tags, their meaning and some colors
from collections import defaultdict

tag_map = {
    'CC': ('Coordin. Conjunction', '#c18401'),
    'TO': ('“to”', '#c18401'),
    'DT': ('Determiner', '#c18401'),
    'UH': ('Interjection', '#c18401'),
    'EX': ('Existential ‘there', '#c18401'),
    'MD': ('Modal can', '#c18401'),
    'LS': ('List item marker', '#c18401'),
    'IN': ('Preposition/sub-conj', '#c18401'),
    'CD': ('Cardinal number', '#282828'),
    'FW': ('Foreign word', '#282828'),
    'NN': ('Noun, singular/mass', '#282828'),
    'NNS': ('Noun, plural', '#282828'),
    'NNP': ('Proper noun, singul.', '#282828'),
    'NNPS': ('Proper noun, plural', '#282828'),
    'JJ': ('Adjective', '#50a14f'),
    'JJR': ('Adj. comparative ', '#50a14f'),
    'JJS': ('Adj. superlative ', '#50a14f'),
    'VB': ('Verb, base form', '#e45649'),
    'VBD': ('Verb, past tense ', '#e45649'),
    'VBG': ('Verb, gerund ', '#e45649'),
    'VBN': ('Verb, past particip. ', '#e45649'),
    'VBP': ('Verb, non-3sg pres', '#e45649'),
    'VBZ': ('Verb, 3sg pres ', '#e45649'),
    'WDT': ('Wh-determiner', '#4078f2'),
    'WP': ('Wh-pronoun', '#4078f2'),
    'WP$': (' Possessive wh-', '#4078f2'),
    'WRB': ('Wh-adverb how', '#4078f2'),
    'PDT': ('Predeterminer ', '#4078f2'),
    'POS': ('Possessive ending', '#4078f2'),
    'PP': ('Personal pronoun', '#4078f2'),
    'PP$': (' Possessive pronoun ', '#4078f2'),
    'RB': ('Adverb', '#a626a4'),
    'RBR': ('Adverb, comparative', '#a626a4'),
    'RBS': ('Adverb, superlative', '#a626a4'),
    'RP': ('Particle', '#a626a4'),
}
tag_map = defaultdict(lambda: ('', '#282828'), tag_map)


def display_pos_tagging(tokens: pd.Series,
                        predicted_tags: pd.Series,
                        correct_tags: pd.Series = None,
                        limit=1000):
    # If no correct tags are passed, we ignore the "error highlighting"
    if correct_tags is None:
        correct_tags = predicted_tags

    # Limit the inputs
    tokens = tokens[:limit]
    predicted_tags = predicted_tags[:limit]
    correct_tags = correct_tags[:limit]

    # Iterate through tokens and tags, generating styled HTML based on the tags
    html_sequence = []
    for token, tag, correct in zip(tokens, predicted_tags, correct_tags):
        tag_meaning = tag_map[tag][0]
        err = 'pos-error' if tag != correct else ''
        h = f'<div class="token {tag} {err}">{token} <span class="tag">[{tag}] {tag_meaning}</span></div>'
        if tag == '.':
            h += '<div class="separator"/>'
        html_sequence.append(h)
    html_body = '<div class="pos-visualizer">'
    html_body += ' '.join(html_sequence) + '</div>'

    # Generate the style (WARNING: CSS lies ahead)
    html_style = """
	<style>
	.pos-visualizer { padding: 32px; background-color: #FEFEFE; border-left:solid 1px grey;}
	.token { position:relative; display:inline-block; font-size:16px;}
	.token .tag { 
		visibility:hidden; width: 120px; text-align:center; position:absolute;
		width: 160px; background-color: #282828; color: #fff; border-radius: 6px;
		z-index: 1; bottom: 100%; left: 50%; margin-left:-80px; font-size:12px;
	}
	.pos-error { text-decoration: underline solid #F94144;}
	.separator { margin-top:12px }
	.token:hover .tag { visibility:visible }
	"""
    html_style += '\n'.join((f'.{tag} {{color:{tag_map[tag][1]};}}'
                             for tag in predicted_tags.unique()))
    html_style += '</style>'

    # Display the HTML in the cell's output
    display(HTML(html_style + html_body))

In [ ]:
# Display some sample POS tagging (hover on text for tag meaning)
predicted_example = dataset['tag'].copy()
predicted_example[0:8] = 'CD'  # Wrong prediction example for the first 8 words
display_pos_tagging(dataset['token'],
                    predicted_example,
                    dataset['tag'],
                    limit=120)

### 1.4 - Pre-processing

#### 1.4.1 - Analysis
First of all, let's check if the GloVe embedded words contains just lowercase words or also capitalized or uppercase ones.

In [ ]:
is_upper = lambda w: not w.islower() and w.isalpha()
glove_keys = glove_embedding.key_to_index.keys()
uppercase_keys = list(filter(is_upper, glove_keys))

# print(uppercase_keys)
print(
    f"There are {len(uppercase_keys)} not lowercase tokens (that do not contain numbers or symbols) in GloVe keys"
)

Our GloVe contains 128 keys that are not lowercase (and are only letters). Upon further inspection, those keys contain only non-latin characters strings, from languages such as Chinese, Arabic and others.

Now we take a look at the uppercase words inside our corpus.

In [ ]:
uppercase_tokens = list(filter(is_upper, dataset['token'].unique()))
print(
    f"There are {len(uppercase_tokens)} uppercase alpha tokens in the corpus")

uppercase_tags = dataset[dataset['token'].str[0].str.isupper()]\
        .groupby('tag')\
        .size()\
        .sort_values(ascending=False)\
        .reset_index(name='capitalized counts')
uppercase_tags.head()  # Display just the head for brevity

The corpus contains about 3000 uppercase (letters only) tokens, which are mostly latin. Those tokens come mostly from proper nouns, determiners, pronouns and generally words that come after a full stop (.)

For example, personal pronouns are meaningfully capitalized only in the case of "I", which is capitalized no matter where it occurs in a sentence.

A couple of weird uppercase tokens are:
* `$`: Dollar mark
* `,`: Non-full stop break punctuation marks for the sentence

Lets see when `$` and `,` are capitalized

In [ ]:
dataset[dataset['tag'] == '$'].groupby('token').size().reset_index(
    name='"$"-tag counts')

The `$` tag is attached not only to the dollar symbol, but also to "C\$" (Canadian dollars) and "US\$" (United States dollar), which are capitalized strings.

In [ ]:
tmp = dataset[dataset['token'].str[0].str.isupper()]
tmp[tmp['tag'] == ',']

What does "Wa" mean? and why is it tagged as `,`? Let's see it in use.

In [ ]:
wa_sentences = dataset[dataset['token'].eq('Wa')]['sentence']
wa_df = dataset[dataset['sentence'].isin(wa_sentences)]
display_pos_tagging(wa_df['token'], wa_df['tag'])

The text snipped above provides an insight on the meaning of the word "Wa". Also, it allows us to see that the single instance in which "Wa" is tagged as `,` is a labeling mistake, as the other occurrences are tagged as `NNP`. 

This outlier also indicates that the corpus contains some misclassified tokens. We can ignore this issue, as we can assume that labeling mistakes are very rare.

Now we want to see which "special" tokens are contained in the embedding. Special tokens include punctuation, quotes, symbols and the `-LRB-`, `-RRB-`, `-LSB-` etc... parenthesis-brackets tokens.

In [ ]:
special_tokens = [
    *',.:;"`$#£!%/?^-()[]{}_', "''", "``", "--", "-LRB-", "-RRB-", "-LSB-",
    "-RSB-", "-LCB-", "-RCB-"
]
for st in special_tokens:
    if st not in glove_keys:
        print(f"GloVe does not contain token {st}")

So the embedding contains most special characters, except the `-XYB-` tokens which are "artifacts" from how the treebank has been constructed. We can safely convert those into

#### 1.4.2 - Text Cleaning
##### Lowercase
We considered a couple of options for **cleaning the dataset from capitalized / uppercase tokens**:  
1. Make ALL tokens lowercase  
2. Make all tokens lowercase, except personal nouns and "I" pronouns.

Both approaches come with some **issues**:
1. By making all tokens lowercase, the model *"loses information"* about proper nouns and other kind of tokens.  
2. This approach keeps information about those tags but has one big caveat: *we modify the test set* using the "correct `tag`", which is information that in real applications wouldn't be available, more so it's the thing we are trying to predict!

Another alternative is, instead of cleaning the dataset beforehand, we can handle the upper and lower cases directly in the model's **embedding**.  
If the GloVe keys were to contain also uppercase words, we could use a two-pass lookup in the Glove (first for the raw token and then for its lowercase version).   
But since the GloVe keys contain only lowercase (latin) words, we can safely do only one pass (looking only at the lowercase word), even though performing two table accesses instead of one does not impact significantly on the performance.  
The main advantage of handling lower/upper cases inside the embedding is that the tokens are not modified, so that we can output the documents in the exact form as they are provided, without the need of keeping a separate data structure for storing the correct capitalized documents.  
Its disadvantage is training speed, since words would need to be converted multiple times during training which slows down the process. 

Overall, the best option in our case is to simply convert the dataset to lowercase, since our objective is just to perform Part Of Speech tagging.

##### Brackets
As said previously, it is safe to convert all `-XYB-` tokens, even inside the test set, since to perform the conversion we are just using the token itself and not the tag.

In [ ]:
# Convert the brackets
for token, bracket in [('-LRB-', '('), ('-RRB-', ')'), ('-LSB-', '['),
                       ('-RSB-', ']'), ('-LCB-', '{'), ('-RCB-', '}')]:
    dataset.loc[dataset.token == token, 'token'] = bracket
    # dataset.where(dataset['token'].ne(token), bracket, inplace=True)

# Convert the dataset tokens into lowercase
dataset.loc[:, 'token'] = dataset['token'].str.lower()

### 1.5 - Splitting
After pre-processing the data, we can finally split the dataset into train, validation and test.

In [ ]:
train_ds = dataset[dataset['document'].lt(100)]
validation_ds = dataset[dataset['document'].between(100, 149)].reset_index()
test_ds = dataset[dataset['document'].gt(149)].reset_index()

print_split = lambda df: f"{df.groupby('document').ngroups} documents, {len(df)} tokens"
print(f"""Dataset split: 
    TRAIN: {print_split(train_ds)}
    VALIDATION: {print_split(validation_ds)}
    TEST: {print_split(test_ds)}
""")

### 1.6 - OOV Handling

#### 1.6.1 - OOV analysis
First of all, lets take a look at how many Out Of Vocabulary terms we have in our dataset

In [ ]:
def get_oov(tokens, embedding_keys):
    return set(tokens) - set(embedding_keys)


oov_train = get_oov(train_ds['token'].unique(), glove_keys)
oov_valid = get_oov(validation_ds['token'].unique(), glove_keys)
oov_test = get_oov(test_ds['token'].unique(), glove_keys)

print_oov = lambda s, d: f"{len(s)} [{len(s) / len(d['token'].unique()) * 100:.2f}%]"
print(f"""Number of OOV tokens in dataset:
    TRAIN: {print_oov(oov_train, train_ds)}
    VALIDATION: {print_oov(oov_valid, validation_ds)}
    TEST: {print_oov(oov_test, test_ds)}
""")

Now we can take a look at the **"incremental"** OOV words, meaning that for the **training** set the OOV remain the *same*, for the **validation** we consider the previously *training-oov-terms as present in the vocabulary*, and finally for the **test** set *both the training and validation oov are present*.

In [ ]:
print(
    f"""Number of OOV tokens in dataset, considering INCREMENTAL OOV EMBEDDING:
    TRAIN: {print_oov(oov_train, train_ds)}
    VALIDATION: {print_oov(oov_valid - oov_train, validation_ds)}
    TEST: {print_oov(oov_test - (oov_valid | oov_train), test_ds)}
    === TOTAL: {print_oov(oov_train | oov_valid | oov_test, dataset)}
""")

#### 1.6.2 - Filling the embedding
We can now fill the GloVe vocabulary with the out of vocabulary tokens. We considered many strategies:
1. Static embedding with the same vector for all OOV tokens (eg zeros)  
2. Random embedding  
3. Estimating the OOV embedding by considering the embedding of neighboring words in the corpus and performing algebraic operations (eg mean)

Since the OOV tokens are not negligible (about 6% of the *total* dataset), we opted for option number 3

NOTE: A slight variation of (3) would be to do a form of "online learning", where after the training OOV terms are computed, their estimate is refined using the data present in the validation/testing dataset, using some sort of weighted average between the current learnt embedding and the estimate using the other dataset. This requires more bookkeeping so for brevity it has not been chosen.


In [ ]:
"""
X Francesco:
ci stanno due funzioni che calcolano la media
- la prima se non ha abbastanza vicini restituisce uno dei due oppure un vettore a caso
- la seconda scorre finchè non trova un embedding valido
TODO: REMOVE THIS COMMENT
"""


def compute_neighbor_mean(oov_token: str, df: pd.DataFrame,
                          embedding: KeyedVectors) -> np.ndarray:
    # Find indexes where the oov token appears, and shift them by -1 +1
    indexes = df.index[df['token'] == oov_token].values
    indexes = np.concatenate((indexes - 1, indexes + 1))
    # Check for out of bounds: (very unnecessary)
    indexes = indexes[(indexes >= 0) | (indexes <= len(df))]

    # Save all neighbors embeddings in a list, only if embedding is found
    neighbor_tokens = df['token'].iloc[indexes]
    neighbor_embeddings = []
    for nt in neighbor_tokens:
        if nt in embedding:
            neighbor_embeddings.append(embedding[nt])

    # If we have enough elements, return their mean
    if len(neighbor_embeddings) > 1:
        return np.mean(neighbor_embeddings, axis=0)

    # If we have just one element, return it
    if len(neighbor_embeddings) == 1:
        return neighbor_embeddings[0]

    # Otherwise, we don't have any encoded neighbors, so we generate a random vector
    print("0 NB", oov_token)
    return np.random.uniform(-0.05, 0.05, size=embedding['0'].size)


def robust_compute_neighbor_mean(oov_token: str, df: pd.DataFrame,
                                 embedding: KeyedVectors) -> np.ndarray:
    # Find indexes where the oov token appears, and shift them by -1 +1
    indexes = df.index[df['token'] == oov_token].values
    indexes = np.concatenate((indexes - 1, indexes + 1))

    # For each oov word index, look at the left and right until a word with embedding has been found
    # AH YES, I LOVE C-LIKE CODE IN PYTHON TODO: REMOVE THIS COMMENT
    neighbor_embeddings = []
    for idx in indexes:
        for direction in (range(idx - 1, -1, -1), range(idx + 1, len(df))):
            for i in direction:
                tok = df['token'].iloc[i]
                if tok not in embedding:
                    continue
                emb = embedding[tok]
                neighbor_embeddings.append(emb)
                break

    return np.mean(neighbor_embeddings, axis=0)


def fill_oov_embedding(oov_tokens: Set, df: pd.DataFrame,
                       embedding: KeyedVectors) -> KeyedVectors:
    # Clone the embedding (KeyedVectors does not have a clone method)
    from copy import deepcopy
    filled = deepcopy(embedding)

    # Estimate the OOV embeddings
    keys, values = [], []
    for oov in oov_tokens:
        estimated_embedding = robust_compute_neighbor_mean(oov, df, filled)
        keys.append(oov)
        values.append(estimated_embedding)
    # print(list(zip(keys, values)))
    # Add the estimates to the embedding
    filled.add_vectors(keys, values)
    return filled


training_embedding = fill_oov_embedding(oov_train, train_ds, glove_embedding)
validation_embedding = fill_oov_embedding(oov_valid - oov_train, validation_ds,
                                          training_embedding)
testing_embedding = fill_oov_embedding(oov_test - oov_valid - oov_train,
                                       test_ds, validation_embedding)


### Vocabulary Creation
> TODO: it would probably be better to define a ``Keras.Tokenizer`` wrapper that handles vocabulary creation, embedding and OOV tokens alltogether, as done is Section 6.3 of `Tutorial2`.

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(filters='', lower=False)
tokenizer.fit_on_texts(train_ds['token'].values)

Save the vocabulary for a more detailed inspection:

In [ ]:
vocab_path = os.path.join(os.getcwd(), 'vocab.json')

with open(vocab_path, 'w') as f:
    simplejson.dump(tokenizer.word_index, f, indent=4)